<a href="https://colab.research.google.com/github/asmaakhaledd/PID-NN/blob/main/opt_of_PID_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install numpy pandas tensorflow control matplotlib xmltodict scikit-learn

In [26]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
import matplotlib.pyplot as plt
from datetime import datetime

Load XML Data

In [27]:
#Automatically loads all XML files (5 training, 5 testing)
train_files = sorted(glob.glob("/content/drive/MyDrive/GP PID/dataset/*-ws-training.xml"))
test_files = sorted(glob.glob("/content/drive/MyDrive/GP PID/dataset/*-ws-testing.xml"))

Check Corrupted XML File

In [28]:
import xml.etree.ElementTree as ET

for file in train_files + test_files:
    try:
        tree = ET.parse(file)
        print(f"✅ {file} is valid")
    except ET.ParseError as e:
        print(f"❌ Error in {file}: {e}")

✅ /content/drive/MyDrive/GP PID/dataset/559-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/563-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/570-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/575-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/588-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/591-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/559-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/563-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/570-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/575-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/588-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/591-ws-testing.xml is valid


Parse XML Data

In [29]:
#Extracts Time, Glucose
#Sorts data by timestamp to maintain correct sequence order
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []

    # Extract patient weight
    weight = float(root.get('weight', 0))  # Defaults to 0 if missing

    # Process glucose levels
    for event in root.find('glucose_level'):
        timestamp = datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S")
        glucose = float(event.get('value'))


        # Append extracted data
        data.append({
            'timestamp': timestamp,
            'glucose': glucose,
            'weight': weight
        })

    df = pd.DataFrame(data)

    # Debugging output
    print(f"Parsed {file_path}, {len(df)} records")
    print(df.head(10))  # Show first 10 rows for debugging

    return df.sort_values('timestamp')

# Load all files
train_dfs = [parse_xml(f) for f in train_files]
test_dfs = [parse_xml(f) for f in test_files]

# Concatenate all data
train_df = pd.concat(train_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)

# Final debug prints
print("\nFinal Train DataFrame:")
print(train_df.head(10))
print("\nFinal Test DataFrame:")
print(test_df.head(10))

Parsed /content/drive/MyDrive/GP PID/dataset/559-ws-training.xml, 10796 records
            timestamp  glucose  weight
0 2021-12-07 01:17:00    101.0    99.0
1 2021-12-07 01:22:00     98.0    99.0
2 2021-12-07 01:27:00    104.0    99.0
3 2021-12-07 01:32:00    112.0    99.0
4 2021-12-07 01:37:00    120.0    99.0
5 2021-12-07 01:42:00    127.0    99.0
6 2021-12-07 01:47:00    135.0    99.0
7 2021-12-07 01:52:00    142.0    99.0
8 2021-12-07 01:57:00    140.0    99.0
9 2021-12-07 02:02:00    145.0    99.0
Parsed /content/drive/MyDrive/GP PID/dataset/563-ws-training.xml, 12124 records
            timestamp  glucose  weight
0 2021-09-13 12:33:00    219.0    99.0
1 2021-09-13 12:38:00    229.0    99.0
2 2021-09-13 12:43:00    224.0    99.0
3 2021-09-13 12:48:00    221.0    99.0
4 2021-09-13 12:53:00    215.0    99.0
5 2021-09-13 12:58:00    209.0    99.0
6 2021-09-13 13:03:00    203.0    99.0
7 2021-09-13 13:08:00    199.0    99.0
8 2021-09-13 13:13:00    196.0    99.0
9 2021-09-13 13:18:00

Convert Time into Numerical Features

In [10]:
# Convert Time into Cyclic Features (Sin/Cos encoding)
def preprocess_time_features(df):
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['time_sin'] = np.sin(2 * np.pi * df['hour'] / 24)  # Cyclic encoding
    df['time_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df.drop(['timestamp', 'hour', 'minute'], axis=1)

train_df = preprocess_time_features(train_df)
test_df = preprocess_time_features(test_df)

Convert all data to float32

In [11]:
train_df = train_df.astype(np.float32)
test_df = test_df.astype(np.float32)

Prepare sequences for the PID tuning model

In [12]:
def prepare_pid_training_data(df, sequence_length=30):
    X_pid, y_pid = [], []
    for i in range(len(df) - sequence_length):
        glucose_error = df['glucose'].iloc[i] - 110  # Setpoint = 110 mg/dL (target glucose)
        glucose_change = df['glucose'].iloc[i] - df['glucose'].iloc[i-1] if i > 0 else 0

        # Kp, Ki, Kd are based on glucose error and change (scaled)
        Kp = 0.05 * np.log(1 + abs(glucose_error))
        Ki = 0.005 * np.log(1 + abs(glucose_error))
        Kd = 0.002 * np.log(1 + abs(glucose_change))

        # Add features (glucose_error, glucose_change, weight, time_sin, time_cos)
        X_pid.append([glucose_error, glucose_change, df['weight'].iloc[i],
                      df['time_sin'].iloc[i], df['time_cos'].iloc[i]])  # Adjust for cyclic time encoding
        y_pid.append([Kp, Ki, Kd])  # Target PID gains

    return np.array(X_pid), np.array(y_pid)

X_pid_train, y_pid_train = prepare_pid_training_data(train_df)
X_pid_test, y_pid_test = prepare_pid_training_data(test_df)


Define LSTM Model for PID Parameter Tuning

In [13]:
pid_input = Input(shape=(X_pid_train.shape[1], 1))
pid_lstm = LSTM(64, activation='tanh', return_sequences=True)(pid_input)
pid_lstm = LSTM(32, activation='tanh')(pid_lstm)
pid_output = Dense(3, activation='linear')(pid_lstm)  # Predict Kp, Ki, Kd

pid_model = Model(inputs=pid_input, outputs=pid_output)
pid_model.compile(optimizer='adam', loss='mse')

Reshape X_pid train  and X_pid test for lstm

In [14]:
X_pid_train_reshaped = X_pid_train.reshape((X_pid_train.shape[0], X_pid_train.shape[1], 1))  # (samples, timesteps, features)
X_pid_test_reshaped = X_pid_test.reshape((X_pid_test.shape[0], X_pid_test.shape[1], 1))  # (samples, timesteps, features)

Train and save the model

In [15]:
pid_model.fit(X_pid_train_reshaped, y_pid_train, epochs=50, batch_size=32, validation_data=(X_pid_test_reshaped, y_pid_test))
pid_model.save("/content/drive/MyDrive/GP PID/opt_pid_tuning_model_2.h5")

Epoch 1/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 3.6265e-04 - val_loss: 4.3331e-06
Epoch 2/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 5.8504e-06 - val_loss: 1.8974e-06
Epoch 3/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 3.6503e-06 - val_loss: 9.1095e-06
Epoch 4/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 2.4788e-06 - val_loss: 9.4465e-07
Epoch 5/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.4918e-06 - val_loss: 1.6501e-06
Epoch 6/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 1.1261e-06 - val_loss: 2.2598e-07
Epoch 7/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 9.9628e-07 - val_loss: 3.5920e-07
Epoch 8/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 9.0869e-07 - val_loss: 8.5362e-07
Epoch 9/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 6.1734e-07 - val_loss: 6.5904e-07
Epoch 10/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 1.0877e-06 - val_loss: 8.6589e-08
Epoch 11/50
2164/2164 ━━━━━━━

Save and Load the model

In [30]:
import tensorflow as tf

# Define the loss function explicitly
pid_model = tf.keras.models.load_model(
    "/content/drive/MyDrive/GP PID/opt_pid_tuning_model_2.h5",
    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
)

# Print model summary to confirm it's loaded
pid_model.summary()
import tensorflow as tf

# Load the model with safe mode
pid_model = tf.keras.models.load_model("/content/drive/MyDrive/GP PID/opt_pid_tuning_model_2.h5", compile=False)

# Compile again with correct loss
pid_model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 5, 1)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 5, 64)               │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,413 (114.90 KB)

 Trainable params: 29,411 (114.89 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)



```
# This is formatted as code
```

Load pid model

In [ ]:
# pid_model = tf.keras.models.load_model("/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5", compile=False)

In [ ]:
#test
pid_gains = pid_model.predict(X_pid_test)

Test the model

In [31]:
# Test the model

num_samples = 50  # Number of test cases to evaluate

for i in range(num_samples):
    print(f"\nSample {i+1}:")

    # Get the actual glucose value from the test dataset (assuming it's part of X_pid_test)
    glucose_level = X_pid_test[i][0]  # Assuming glucose is the first feature in the input sequence

    # Get the corresponding timestamp from test_df
    timestamp = test_df['timestamp'].iloc[i]  # 'timestamp' should be in the DataFrame after parsing

    # Weight of the patient
    weight = test_df['weight'].iloc[i]

    # Calculate Basal Insulin Dosage:
    basal_rate_per_kg = 0.5  # Adjust this according to patient-specific needs

    # Total Daily Insulin (TDI) requirement (approximation)
    TDI = basal_rate_per_kg * weight  # Total daily insulin in units (for basal)

    # Basal Insulin is typically 50% of TDI for basal rate
    basal_insulin_dosage = 0.5 * TDI  # Total daily basal insulin

    # Hourly basal rate (divide by 24 to get hourly insulin)
    hourly_basal_rate = basal_insulin_dosage / 24  # Units per hour for basal insulin

    # Real-time Adjustment: Adjust insulin more significantly if glucose is above target
    target_glucose = 110  # mg/dL (target glucose level)

    if glucose_level > target_glucose:
        # Increase basal insulin more substantially as glucose increases
        adjustment_factor = (glucose_level - target_glucose) / 100  # Adjust based on glucose deviation
        adjusted_basal_rate = hourly_basal_rate + adjustment_factor
    elif glucose_level < target_glucose:
        # If glucose is below target, decrease basal insulin by a smaller factor
        adjustment_factor = (target_glucose - glucose_level) / 100
        adjusted_basal_rate = hourly_basal_rate - adjustment_factor
    else:
        # If glucose is at target, keep basal insulin as is
        adjusted_basal_rate = hourly_basal_rate

    # **Apply basal rate limits**:
    min_basal_rate = 0.3  # Minimum safe basal rate in units per hour
    max_basal_rate = 1.5  # Maximum safe basal rate in units per hour

    # Ensure the adjusted basal rate is within the defined limits
    adjusted_basal_rate = max(min_basal_rate, min(adjusted_basal_rate, max_basal_rate))

    # Print results
    print(f"Timestamp: {timestamp}")
    print(f"Glucose: {glucose_level:.2f} mg/dL")
    print(f"Adjusted Hourly Basal Insulin: {adjusted_basal_rate:.2f} U per hour")
    print(f"Weight: {weight} kg")

    print("-" * 50)



Sample 1:
Timestamp: 2022-01-18 00:01:00
Glucose: 69.00 mg/dL
Adjusted Hourly Basal Insulin: 0.62 U per hour
Weight: 99.0 kg
--------------------------------------------------

Sample 2:
Timestamp: 2022-01-18 00:06:00
Glucose: 73.00 mg/dL
Adjusted Hourly Basal Insulin: 0.66 U per hour
Weight: 99.0 kg
--------------------------------------------------

Sample 3:
Timestamp: 2022-01-18 00:11:00
Glucose: 77.00 mg/dL
Adjusted Hourly Basal Insulin: 0.70 U per hour
Weight: 99.0 kg
--------------------------------------------------

Sample 4:
Timestamp: 2022-01-18 00:16:00
Glucose: 81.00 mg/dL
Adjusted Hourly Basal Insulin: 0.74 U per hour
Weight: 99.0 kg
--------------------------------------------------

Sample 5:
Timestamp: 2022-01-18 00:21:00
Glucose: 85.00 mg/dL
Adjusted Hourly Basal Insulin: 0.78 U per hour
Weight: 99.0 kg
--------------------------------------------------

Sample 6:
Timestamp: 2022-01-18 00:26:00
Glucose: 89.00 mg/dL
Adjusted Hourly Basal Insulin: 0.82 U per hour
Weigh

In [24]:
print(test_df.columns)  # This will show the actual column names in your DataFrame


Index(['glucose', 'weight', 'time_sin', 'time_cos'], dtype='object')
